In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread("mod_img11.jpg")
img3 = np.copy(img)
img2 = img
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
grey = cv2.GaussianBlur(grey,(17,17), 0)
th = cv2.adaptiveThreshold(grey,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,23,2)
kernel = np.ones((1,5),np.uint8)

th = 255 - th

nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(th, None, None, None, 4, cv2.CV_32S)
areas = stats[1:,cv2.CC_STAT_AREA]
result = np.zeros((labels.shape), np.uint8)
for i in range(0, nlabels - 1):
    if areas[i] >= 210:   #keep
        result[labels == i + 1] = 255

#cv2.imshow("Binary", binary_map)
result = 255-result



cv2.imwrite("inter1.jpg", result)

True

In [3]:
contours, hierarchy = cv2.findContours(result, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
centroid = set()

for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
i = 0
others = list()
while i<len(boundRect):
    if boundRect[i][2] > 25 and boundRect[i][3] > 25:
        i += 1
    else:
        temp = boundRect[i]
        boundRect.remove(boundRect[i])
        if temp[2] >5 and temp[3] >5 :         
            others.append(temp)
        
boundRect.sort(key = lambda x:x[0])
i = 0

while i < len(boundRect):
    centroid = (boundRect[i][0] + boundRect[i][2]/2, boundRect[i][1] + boundRect[i][3]/2)
    j = i+1 
    while j < len(boundRect):
        if abs(boundRect[j][0]+boundRect[j][2]/2 -centroid[0])<5 and abs(boundRect[j][1]+boundRect[j][3]/2 -centroid[1])<5: 
            boundRect.remove(boundRect[j])
            break
        else:
                j+=1
    i += 1

for i in boundRect:
    if i[2]==img3.shape[1] and i[3]==img3.shape[0]:
        boundRect.remove(i)
        break
print(boundRect)

[(63, 100, 512, 31), (98, 243, 500, 80), (102, 428, 497, 79), (104, 60, 31, 39), (110, 443, 128, 54), (119, 625, 647, 273), (129, 190, 30, 28), (131, 647, 298, 90), (135, 725, 301, 98), (139, 807, 300, 82), (154, 375, 31, 30), (175, 580, 57, 37), (242, 440, 126, 55), (366, 439, 228, 53), (437, 715, 316, 84), (468, 46, 33, 44), (729, 213, 503, 86), (736, 396, 44, 46), (742, 510, 47, 43), (819, 503, 31, 38), (828, 392, 45, 36), (861, 504, 43, 33), (871, 161, 27, 41), (876, 602, 277, 86), (880, 721, 330, 102), (979, 380, 59, 43), (982, 626, 28, 32), (986, 757, 29, 28), (1066, 377, 38, 34), (1154, 238, 46, 29)]


In [4]:
boundRect.sort(key = lambda x:x[2])

fields={}

for i in range(len(boundRect)):
    x1,y1,w1,h1=boundRect[i][0:]
    fields[(x1,y1,w1,h1)]=[]
        
for i in range(len(boundRect)):
    x1,y1,w1,h1=boundRect[i][0:]
    for big_box in fields:
            x2,y2,w2,h2=big_box[0:]
            if y1>y2 and x1>x2 and y1+h1 < y2+h2 and x1+w1 < x2+w2:
                fields[big_box].append((x1,y1,w1,h1))
                break
                
                
for c in fields:
    if(len(fields[c]) >=1):
        for i in range(len(fields[c])):
            boundRect.remove(fields[c][i])
        

for c in boundRect:
    cv2.rectangle(img2, (int(c[0]), int(c[1])), \
          (int(c[0]+c[2]), int(c[1]+c[3])), (0,0,255), 2)
cv2.imwrite("inter6.jpg", img2)  

True